# Splice-junction Gene Sequences Data Set 


In [6]:

#Imports
%matplotlib inline
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit

import tensorflow as tf
from tensorflow.contrib import learn
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib


### Load the dataset

In [7]:
df = pd.read_csv("..\data\splice.data", header=None)
df.columns = ['classlabel', 'name', 'sequence']
df.tail()


,classlabel,name,sequence
3185,N,ORAHBPSBD-NEG-2881,TCTCTTCCCTTCCCCTCTCTCTTTCTTTCTTTT...
3186,N,ORAINVOL-NEG-2161,GAGCTCCCAGAGCAGCAAGAGGGCCAGCTGAA...
3187,N,ORARGIT-NEG-241,TCTCGGGGGCGGCCGGCGCGGCGGGGAGCG...
3188,N,TARHBB-NEG-541,ATTCTACTTAGTAAACATAATTTCTTGTG...
3189,N,TARHBD-NEG-1981,AGGCTGCCTATCAGAAGGTGGTGGCTGGTG...


In [14]:
# Encoding class labels
# We don't apply one hot encoding to y, because TF DNN model has already supported it internally.

class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)

print(y)
print(y.shape)

[0 0 0 ..., 2 2 2]
(3190,)


In [23]:
# Encoding sequence
# Here we use one hot encoding to encode the character in DNA sequence. 
# So each dna sequence is converted to a 60x8 2D array 

# D: A or G or T 
# N: A or G or C or T 
# S: C or G 
# R: A or G

def Seq2Vec(seq):
    s = str(seq).strip()
    
    if( False ) :
        CharDict = { "A":[0,0,0,1],
                     "G":[0,0,1,0],
                     "C":[0,1,0,0],
                     "T":[1,0,0,0],
                     "D":[1,0,1,1],
                     "N":[1,1,1,1],
                     "S":[0,1,1,0],
                     "R":[0,0,1,1]}
    else : 
        CharDict = { "A":[0.,0.,0.,0.,0.,0.,0.,1.],
                     "G":[0.,0.,0.,0.,0.,0.,1.,0.],
                     "C":[0.,0.,0.,0.,0.,1.,0.,0.],
                     "T":[0.,0.,0.,0.,1.,0.,0.,0.],
                     "D":[0.,0.,0.,1.,0.,0.,0.,0.],
                     "N":[0.,0.,1.,0.,0.,0.,0.,0.],
                     "S":[0.,1.,0.,0.,0.,0.,0.,0.],
                     "R":[1.,0.,0.,0.,0.,0.,0.,0.]}
    
    return np.asarray([CharDict[c] for c in s]).flatten()
   
df['seqvec'] = df['sequence'].apply(Seq2Vec)
X = df['seqvec'].values
X = np.vstack(X)

print(df['sequence'][0])
#print(X[0])
print(X.shape)

               CCAGCTGCATCACAGGAGGCCAGCGAGCAGGTCTGTTCCAAGGGCCTTCGAGCCAGTCTG
(3190, 480)


In [24]:
# Split the data set into training/test set

sss = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=0)
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
print("training data size:", X_train.shape, "; test data size:", X_test.shape )
print("training y size:", y_train.shape, "; test y size:", y_test.shape )

training data size: (2552, 480) ; test data size: (638, 480)
training y size: (2552,) ; test y size: (638,)


In [25]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  # DNA slice has 60x8, and have one color channel
  input_layer = tf.reshape(features, [-1, 60, 8, 1])

  conv1_depth = 16
  conv2_depth = 32

  # Convolutional Layer #1
  # Computes 32 features using a 3x3 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 60, 8, 1]
  # Output Tensor Shape: [batch_size, 60, 8, 16]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=conv1_depth,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 60, 8, 16]
  # Output Tensor Shape: [batch_size, 30, 4, 16]
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2
  # Computes 64 features using a 3x3 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 30, 4, 16]
  # Output Tensor Shape: [batch_size, 30, 4, 32]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=conv2_depth,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #2
  # Second max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 30, 4, 32]
  # Output Tensor Shape: [batch_size, 15, 2, 32]
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 15, 2, 32]
  # Output Tensor Shape: [batch_size, 15 * 2 * 32]
  pool2_flat = tf.reshape(pool2, [-1, 15 * 2 * conv2_depth])

  # Dense Layer
  # Densely connected layer with 1024 neurons
  # Input Tensor Shape: [batch_size, 15 * 2 * 32]
  # Output Tensor Shape: [batch_size, 512]
  dense = tf.layers.dense(inputs=pool2_flat, units=512, activation=tf.nn.relu)

  # Add dropout operation; 0.6 probability that element will be kept
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == learn.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 512]
  # Output Tensor Shape: [batch_size, 3]
  logits = tf.layers.dense(inputs=dropout, units=3)

  loss = None
  train_op = None

  # Calculate Loss (for both TRAIN and EVAL modes)
  if mode != learn.ModeKeys.INFER:
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=3)
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == learn.ModeKeys.TRAIN:
    train_op = tf.contrib.layers.optimize_loss(
        loss=loss,
        global_step=tf.contrib.framework.get_global_step(),
        learning_rate=0.001,
        optimizer="SGD")

  # Generate Predictions
  predictions = {
      "classes": tf.argmax(
          input=logits, axis=1),
      "probabilities": tf.nn.softmax(
          logits, name="softmax_tensor")
  }

  # Return a ModelFnOps object
  return model_fn_lib.ModelFnOps(
      mode=mode, predictions=predictions, loss=loss, train_op=train_op)

In [26]:
def main(unused_argv):
  # Load training and eval data
  train_data = np.asarray(X_train, dtype=np.float32)  # Returns np.array
  train_labels = np.asarray(y_train, dtype=np.int32)
  eval_data = np.asarray(X_test, dtype=np.float32)  # Returns np.array
  eval_labels = np.asarray(y_test, dtype=np.int32)

  # Create the Estimator
  DNASeq_classifier = learn.Estimator(
      model_fn=cnn_model_fn, model_dir="./tmp/DNASeq_convnet_model")

  # Set up logging for predictions
  # Log the values in the "Softmax" tensor with label "probabilities"
  tensors_to_log = {"probabilities": "softmax_tensor"}
  logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)

  # Train the model
  DNASeq_classifier.fit(
      x=train_data,
      y=train_labels,
      batch_size=100,
      steps=20000,
      monitors=[logging_hook])

  # Configure the accuracy metric for evaluation
  metrics = {
      "accuracy":
          learn.MetricSpec(
              metric_fn=tf.metrics.accuracy, prediction_key="classes"),
  }

  # Evaluate the model and print results
  eval_results = DNASeq_classifier.evaluate(
      x=eval_data, y=eval_labels, metrics=metrics)
  print(eval_results)

In [27]:
if __name__ == "__main__":
  tf.app.run()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_keep_checkpoint_max': 5, '_model_dir': None, '_task_type': None, '_save_checkpoints_secs': 600, '_task_id': 0, '_num_ps_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, '_master': '', '_save_summary_steps': 100, '_evaluation_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002180CCFC160>, '_save_checkpoints_steps': None, '_num_worker_replicas': 0, '_is_chief': True, '_environment': 'local'}
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by mov

C:\Users\qli45\AppData\Local\Continuum\Anaconda3\lib\site-packages\tensorflow\python\util\deprecation.py:248: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./tmp/DNASeq_convnet_model\model.ckpt-31001
INFO:tensorflow:Saving checkpoints for 31002 into ./tmp/DNASeq_convnet_model\model.ckpt.
INFO:tensorflow:step = 31002, loss = 0.0507466
INFO:tensorflow:probabilities = [[ 0.00065099  0.00125843  0.99809057]
 [ 0.96610415  0.0001652   0.03373069]
 [ 0.00020113  0.00001521  0.99978369]
 [ 0.01669035  0.00336814  0.97994149]
 [ 0.02612661  0.62143588  0.3524375 ]
 [ 0.00077959  0.0000834   0.99913698]
 [ 0.00402376  0.99343044  0.0025458 ]
 [ 0.00035435  0.99005836  0.00958736]
 [ 0.0007751   0.00031151  0.99891341]
 [ 0.99852055  0.00002009  0.00145937]
 [ 0.0002592   0.00104085  0.9986999 ]
 [ 0.0132702   0.00778236  0.9789474 ]
 [ 0.00066805  0.00009737  0.99923456]
 [ 0.00046752  0.00130645  0.99822599]
 [ 0.00023979  0.99163812  0.00812219]
 [ 0.00060974  0.00388915  0.9955011 ]
 [ 0.00024502  0.00249501  0.99725997]
 [ 0.02391122  0.07910496  0.8969838 ]


KeyboardInterrupt: 